# Import Libraries

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document
import time

## Initialization

In [2]:
# Get API Keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
# index = pc.Index("echo-openai")

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)

C:\Users\Zeck Lowe\AppData\Local\Temp\ipykernel_14552\1551298541.py:10: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# Chatbot

In [17]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

docsearch = PineconeVectorStore(
    index_name=input("Give organization name"),
    embedding=EMBEDDINGS,
    namespace=input("Give meeting title")
)

retriever = docsearch.as_retriever()

LLM = ChatOpenAI(temperature=0.0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

combine_docs_chain = create_stuff_documents_chain(
    LLM, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(
    retriever, combine_docs_chain
)

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\langsmith\client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Query

In [12]:
query1 = "What was the QA perspective on the kickoff meeting?"

query2 = "What are the key points of the meeting?"

### Query without relevant knowledge

In [13]:
answer1_without_knowledge = LLM.invoke(query1)

print("Query 1:", query1)
print("\nAnswer without knowledge:\n\n", answer1_without_knowledge.content)
print("\n")
time.sleep(2)

Query 1: What was the QA perspective on the kickoff meeting?

Answer without knowledge:

 I don't have specific details about a particular kickoff meeting as my responses are generated based on a mixture of licensed data, data created by human trainers, and publicly available information. I can, however, provide a general overview of what a Quality Assurance (QA) perspective might focus on during a kickoff meeting:

1. **Understanding Requirements**: QA professionals would be keen on understanding the project requirements clearly. This includes knowing what the expected outcomes are, the scope of the project, and specific details about functionalities and user expectations.

2. **Clarifying Test Objectives**: They would want to discuss and clarify the objectives of testing for the project. This involves understanding what needs to be tested, how it should be tested, and the criteria for passing or failing.

3. **Resource Allocation**: From a QA perspective, it's important to discuss th

### Query with Knowledge

In [ ]:
answer1_with_knowledge = retrieval_chain.invoke({"input": query2})

print("Query 1:", query2)
print("\nAnswer with knowledge:\n\n", answer1_with_knowledge['answer'])
print("\nContext used:\n\n", answer1_with_knowledge['context'])
print("\n")
time.sleep(2)

Query 1: What are the key points of the meeting?

Answer with knowledge:

 The key points of the meeting include:

1. **Development Update**: Shaundyl reported that all major features have been implemented and he is currently working on the final round of bug fixes, which should be completed by the next day. However, the QA team needs to review the fixes afterward.

2. **Project Timeline**: Gian confirmed that the project is on track, although there was a slight delay with the marketing materials. He adjusted the social media schedule to compensate for this delay. He also suggested allocating some buffer time for any unforeseen issues as the launch date approaches.

3. **Testing**: Shaundyl plans to finish the bug fixes by midday the following day, allowing the QA team to start their final round of testing immediately after. This schedule provides 24 hours for testing before the product goes live. Shaundyl also committed to being available for any necessary hotfixes during the launch.
